In [97]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [98]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Louis,SN,16.3333,-15.0000,98.24,15,7,10.69,clear sky
1,1,Punta Arenas,CL,-53.1500,-70.9167,53.71,40,75,19.57,broken clouds
2,2,Saint Anthony,US,45.0205,-93.2180,33.69,51,1,8.05,clear sky
3,3,Kungurtug,RU,50.5994,97.5228,-4.74,98,50,1.81,scattered clouds
4,4,Mataura,NZ,-46.1927,168.8643,37.47,99,86,1.77,overcast clouds


In [99]:
# Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [100]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Namatanai,PG,-3.6667,152.4333,78.66,83,52,3.74,moderate rain
6,6,Angoche,MZ,-16.2325,39.9086,77.67,61,76,14.27,broken clouds
9,9,Acapulco,MX,16.8634,-99.8901,81.81,68,64,1.77,broken clouds
10,10,Port Blair,IN,11.6667,92.7500,81.07,80,12,10.16,few clouds
12,12,Rikitea,PF,-23.1203,-134.9692,76.26,75,2,16.55,light rain


In [101]:
# Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                614
City                   614
Country                605
Lat                    614
Lng                    614
Max Temp               614
Humidity               614
Cloudiness             614
Wind Speed             614
Current Description    614
dtype: int64

In [102]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City_ID                605
City                   605
Country                605
Lat                    605
Lng                    605
Max Temp               605
Humidity               605
Cloudiness             605
Wind Speed             605
Current Description    605
dtype: int64

In [103]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Namatanai,PG,78.66,moderate rain,-3.6667,152.4333,
6,Angoche,MZ,77.67,broken clouds,-16.2325,39.9086,
9,Acapulco,MX,81.81,broken clouds,16.8634,-99.8901,
10,Port Blair,IN,81.07,few clouds,11.6667,92.7500,
12,Rikitea,PF,76.26,light rain,-23.1203,-134.9692,
15,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,
19,Balabac,PH,81.27,overcast clouds,7.9868,117.0645,
20,Tshikapa,CD,77.52,broken clouds,-6.4167,20.8000,
25,Bambous Virieux,MU,78.19,few clouds,-20.3428,57.7575,
27,Howard Springs,AU,76.91,broken clouds,-12.4970,131.0470,


In [104]:
# Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [105]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!= ""]
clean_hotel_df.count()

City                   561
Country                561
Max Temp               561
Current Description    561
Lat                    561
Lng                    561
Hotel Name             561
dtype: int64

In [107]:
# Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [113]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [114]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))